# Evaluate model endpoints using Prompt Flow Eval APIs

## Objective

This tutorial provides a step-by-step guide on how to evaluate prompts against variety of model endpoints deployed on Azure AI Platform or non Azure AI platforms. 

This guide uses Python Class as an application target which is passed to Evaluate API provided by PromptFlow SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [promptflow-evals](https://microsoft.github.io/promptflow/reference/python-library-reference/promptflow-evals/promptflow.html)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using promptflow-evals

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [ ]:
%pip install promptflow-evals
%pip install promptflow-azure

### Parameters and imports

In [6]:
from pprint import pprint

import pandas as pd
import random

#### Note Models as a Service (Serverless API) have key-auth only
https://learn.microsoft.com/en-us/azure/ai-studio/how-to/model-catalog-overview#models-as-a-service-pay-as-you-go

## Target Application

We will use Evaluate API provided by Prompt Flow SDK. It requires a target Application or python Function, which handles a call to LLMs and retrieve responses. 

In the notebook, we will use an Application Target `ModelEndpoints` to get answers from multiple model endpoints against provided question aka prompts. 

This application target requires list of model endpoints and their authentication keys. For simplicity, we have provided them in the `env_var` variable which is passed into init() function of `ModelEndpoints`.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv('../.env', override=True)


True

In [2]:
env_var = {
    "gpt4o": {
        "endpoint": os.environ.get("GPT4_ENDPOINT"),
        "key": os.environ.get("GPT4_KEY"),
    },
    "gpt35-turbo": {
        "endpoint": os.environ.get("GPT35_ENDPOINT"),
        "key": os.environ.get("GPT35_KEY"),
    },
    "mistral7b": {
        "endpoint": os.environ.get("MISTRAL7B_ENDPOINT"),
        "key": os.environ.get("MISTRAL7B_KEY"),
    },
    "phi3_serverless": {
        "endpoint": os.environ.get("PHI3_ENDPOINT"),
        "key": os.environ.get("PHI3_KEY"),
    }
}


Please provide Azure AI Project details so that traces and eval results are pushing in the project in Azure AI Studio.

In [4]:
# define the Azure AI Studio connection
azure_ai_project = {
    "subscription_id": os.environ.get("SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("RESOURCE_GROUP_NAME"),
    "project_name": os.environ.get("PROJECT_NAME")
}

## Data

Following code reads Json file "data.jsonl" which contains inputs to the Application Target function. It provides question, context and ground truth on each line. 

In [7]:
df = pd.read_json("../data/data.jsonl", lines=True)
print(df.head())

                                        question  \
0                 What is the capital of France?   
1             Which tent is the most waterproof?   
2           Which camping table is the lightest?   
3  How much does TrailWalker Hiking Shoes cost?    

                                             context  \
0                   France is the country in Europe.   
1  #TrailMaster X4 Tent, price $250,## BrandOutdo...   
2  #BaseCamp Folding Table, price $60,## BrandCam...   
3  #TrailWalker Hiking Shoes, price $110## BrandT...   

                                        ground_truth  
0                                              Paris  
1  The TrailMaster X4 tent has a rainfly waterpro...  
2  The BaseCamp Folding Table has a weight of 15 lbs  
3    The TrailWalker Hiking Shoes are priced at $110  


## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [8]:
from promptflow.core import AzureOpenAIModelConfiguration

# configuration = AzureOpenAIModelConfiguration(
#     azure_endpoint="https://ai-***.openai.azure.com",
#     api_key="**",
#     api_version="2023-03-15-preview",
#     azure_deployment="gpt-35-turbo-16k",
# )
configuration = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    azure_deployment=os.environ.get("AZURE_OPENAI_EVALUATION_DEPLOYMENT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

## Run the evaluation

The Following code runs Evaluate API and uses Content Safety, Relevance and Coherence Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which can route the calls to specific model endpoints using model name in conditional logic.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

In [ ]:
# pip install azure-ai-inference
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

api_key = os.getenv("AZURE_INFERENCE_CREDENTIAL", '')
if not api_key:
  raise Exception("A key should be provided to invoke the endpoint")

client = ChatCompletionsClient(
    endpoint='https://Mistral-large-2407-lqkzb.eastus2.models.ai.azure.com',
    credential=AzureKeyCredential(api_key)
)

model_info = client.get_model_info()
print("Model name:", model_info.model_name)
print("Model type:", model_info.model_type)
print("Model provider name:", model_info.model_provider_name)

payload = {
  "messages": [
    {
      "role": "user",
      "content": "I am going to Paris, what should I see?"
    },
    {
      "role": "assistant",
      "content": "Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world."
    },
    {
      "role": "user",
      "content": "What is so great about #1?"
    }
  ],
  "max_tokens": 4096,
  "temperature": 0.7,
  "top_p": 1
}
response = client.complete(payload)

print("Response:", response.choices[0].message.content)
print("Model:", response.model)
print("Usage:")
print("	Prompt tokens:", response.usage.prompt_tokens)
print("	Total tokens:", response.usage.total_tokens)
print("	Completion tokens:", response.usage.completion_tokens)

In [17]:
from typing import TypedDict
import requests

In [18]:
def query(endpoint: str, headers: str, payload: str) -> str:
    response = requests.post(url=endpoint, headers=headers, json=payload)
    return response.json()

In [19]:
def call_mistral_endpoint(question: str) -> TypedDict:
    endpoint = env_var["mistral7b"]["endpoint"]
    key = env_var["mistral7b"]["key"]

    headers = {"Content-Type": "application/json", "Authorization": ("Bearer " + key)}

    payload = {"messages": [{"content": question, "role": "user"}], "max_tokens": 4096}

    output = query(endpoint=endpoint, headers=headers, payload=payload)
    answer = output["choices"][0]["message"]["content"]
    return {"question": question, "answer": answer}

In [21]:
call_mistral_endpoint("What is the capital of France?")

KeyError: 'choices'

In [11]:
from app_target import ModelEndpoints
import pathlib

from promptflow.evals.evaluate import evaluate
from promptflow.evals.evaluators import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)


content_safety_evaluator = ContentSafetyEvaluator(project_scope=azure_ai_project)
relevance_evaluator = RelevanceEvaluator(model_config=configuration)
coherence_evaluator = CoherenceEvaluator(model_config=configuration)
groundedness_evaluator = GroundednessEvaluator(model_config=configuration)
fluency_evaluator = FluencyEvaluator(model_config=configuration)
similarity_evaluator = SimilarityEvaluator(model_config=configuration)

models = [
    "mistral7b",
]

#path = str(pathlib.Path(pathlib.Path.cwd())) + "/data.jsonl"
path = "/home/krbock/GitHub/rai-genai-workshop/data/data.jsonl"

for model in models:
    randomNum = random.randint(1111, 9999)
    results = evaluate(
        azure_ai_project=azure_ai_project,
        evaluation_name="Eval-Run-" + str(randomNum) + "-" + model.title(),
        data=path,
        target=ModelEndpoints(env_var, model),
        evaluators={
            "content_safety": content_safety_evaluator,
            "coherence": coherence_evaluator,
            "relevance": relevance_evaluator,
            "groundedness": groundedness_evaluator,
            "fluency": fluency_evaluator,
            "similarity": similarity_evaluator,
        },
        evaluator_config={
            "content_safety": {"question": "${data.question}", "answer": "${target.answer}"},
            "coherence": {"answer": "${target.answer}", "question": "${data.question}"},
            "relevance": {"answer": "${target.answer}", "context": "${data.context}", "question": "${data.question}"},
            "groundedness": {
                "answer": "${target.answer}",
                "context": "${data.context}",
                "question": "${data.question}",
            },
            "fluency": {"answer": "${target.answer}", "context": "${data.context}", "question": "${data.question}"},
            "similarity": {"answer": "${target.answer}", "context": "${data.context}", "question": "${data.question}"},
        },
    )

[2024-08-30 16:17:34 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Upload run to cloud: True


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=app_target_modelendpoints_2wpicrpj_20240830_161734_481826
You can view the traces in azure portal since trace destination is set to: azureml://subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635. The link will be printed once the run is finished.


[2024-08-30 16:17:37 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run app_target_modelendpoints_2wpicrpj_20240830_161734_481826, log path: /home/krbock/.promptflow/.runs/app_target_modelendpoints_2wpicrpj_20240830_161734_481826/logs.txt
[2024-08-30 16:17:41 +1000][promptflow._sdk._orchestrator.run_submitter][WARNING] - 4 out of 4 runs failed in batch run.
 Please check out /home/krbock/.promptflow/.runs/app_target_modelendpoints_2wpicrpj_20240830_161734_481826 for more details.


2024-08-30 16:17:40 +1000   72134 execution.bulk     INFO     Process 72159 terminated.
2024-08-30 16:17:40 +1000   72134 execution.bulk     WARNING  Process 72180 had been terminated.
2024-08-30 16:17:40 +1000   72134 execution.bulk     WARNING  Process 72171 had been terminated.
2024-08-30 16:17:40 +1000   72134 execution.bulk     WARNING  Process 72165 had been terminated.


[2024-08-30 16:17:42 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Uploading run 'app_target_modelendpoints_2wpicrpj_20240830_161734_481826' to cloud...
[2024-08-30 16:17:53 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Updating run 'app_target_modelendpoints_2wpicrpj_20240830_161734_481826' portal url to 'https://ai.azure.com/projectflows/trace/run/app_target_modelendpoints_2wpicrpj_20240830_161734_481826/details?wsid=/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourcegroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635'.


Portal url: https://ai.azure.com/projectflows/trace/run/app_target_modelendpoints_2wpicrpj_20240830_161734_481826/details?wsid=/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourcegroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635
2024-08-30 16:17:37 +1000   57173 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-08-30 16:17:37 +1000   57173 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2024-08-30 16:17:37 +1000   57173 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 4}.
2024-08-30 16:17:39 +1000   57173 execution.bulk     INFO     Process name(ForkProcess-40:3)-Process id(72171)-Line number(0) start execution.
2024-08-30 16:17:39 +1000   57

ValueError: Missing required inputs for evaluator content_safety : ['answer'].

In [10]:
env_var

{'gpt4o': {'endpoint': 'https://mssecureai4034688619.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2023-03-15-preview',
  'key': 'b1faabe0884343d78d77a4534662f677'},
 'gpt35-turbo': {'endpoint': 'https://mssecureai4034688619.openai.azure.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-03-15-preview',
  'key': 'b1faabe0884343d78d77a4534662f677'},
 'mistral7b': {'endpoint': 'https://Mistral-large-2407-lqkzb.eastus2.models.ai.azure.com',
  'key': 'Hmzq8a1DpeK2rBgxW9f7Qf4bpHck6GxH'},
 'phi3_serverless': {'endpoint': 'https://Phi-3-small-128k-instruct-akbcg.eastus2.models.ai.azure.com',
  'key': 'rZaWPXQ4zI0zSMLcxdf2Vw3Rqcv9gWgl'}}

In [9]:
from app_target import ModelEndpoints
import pathlib

from promptflow.evals.evaluate import evaluate
from promptflow.evals.evaluators import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)


content_safety_evaluator = ContentSafetyEvaluator(project_scope=azure_ai_project)
relevance_evaluator = RelevanceEvaluator(model_config=configuration)
coherence_evaluator = CoherenceEvaluator(model_config=configuration)
groundedness_evaluator = GroundednessEvaluator(model_config=configuration)
fluency_evaluator = FluencyEvaluator(model_config=configuration)
similarity_evaluator = SimilarityEvaluator(model_config=configuration)

models = [
    "gpt4-0613",
    "gpt35-turbo",
    "mistral7b",
    "phi3_serverless"
]

#path = str(pathlib.Path(pathlib.Path.cwd())) + "/data.jsonl"
path = "/home/krbock/GitHub/rai-genai-workshop/data/data.jsonl"

for model in models:
    randomNum = random.randint(1111, 9999)
    results = evaluate(
        azure_ai_project=azure_ai_project,
        evaluation_name="Eval-Run-" + str(randomNum) + "-" + model.title(),
        data=path,
        target=ModelEndpoints(env_var, model),
        evaluators={
            "content_safety": content_safety_evaluator,
            "coherence": coherence_evaluator,
            "relevance": relevance_evaluator,
            "groundedness": groundedness_evaluator,
            "fluency": fluency_evaluator,
            "similarity": similarity_evaluator,
        },
        evaluator_config={
            "content_safety": {"question": "${data.question}", "answer": "${target.answer}"},
            "coherence": {"answer": "${target.answer}", "question": "${data.question}"},
            "relevance": {"answer": "${target.answer}", "context": "${data.context}", "question": "${data.question}"},
            "groundedness": {
                "answer": "${target.answer}",
                "context": "${data.context}",
                "question": "${data.question}",
            },
            "fluency": {"answer": "${target.answer}", "context": "${data.context}", "question": "${data.question}"},
            "similarity": {"answer": "${target.answer}", "context": "${data.context}", "question": "${data.question}"},
        },
    )

[2024-08-30 15:41:57 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Upload run to cloud: True


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=n1_model_20240830_154157_098890
You can view the traces in azure portal since trace destination is set to: azureml://subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635. The link will be printed once the run is finished.


[2024-08-30 15:42:00 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run n1_model_20240830_154157_098890, log path: /home/krbock/.promptflow/.runs/n1_model_20240830_154157_098890/logs.txt


2024-08-30 15:42:09 +1000   57501 execution.bulk     INFO     Process 57531 terminated.
2024-08-30 15:42:09 +1000   57501 execution.bulk     WARNING  Process 57525 had been terminated.
2024-08-30 15:42:09 +1000   57501 execution.bulk     WARNING  Process 57521 had been terminated.
2024-08-30 15:42:09 +1000   57501 execution.bulk     WARNING  Process 57540 had been terminated.


[2024-08-30 15:42:10 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Uploading run 'n1_model_20240830_154157_098890' to cloud...
[2024-08-30 15:42:21 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Updating run 'n1_model_20240830_154157_098890' portal url to 'https://ai.azure.com/projectflows/trace/run/n1_model_20240830_154157_098890/details?wsid=/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourcegroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635'.


...

[2024-08-30 15:45:38 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run app_target_modelendpoints_75ml9r66_20240830_154535_138482, log path: /home/krbock/.promptflow/.runs/app_target_modelendpoints_75ml9r66_20240830_154535_138482/logs.txt


2024-08-30 15:45:41 +1000   64458 execution.bulk     INFO     Process 64483 terminated.
2024-08-30 15:45:41 +1000   64458 execution.bulk     WARNING  Process 64503 had been terminated.
2024-08-30 15:45:41 +1000   64458 execution.bulk     WARNING  Process 64496 had been terminated.
2024-08-30 15:45:41 +1000   64458 execution.bulk     WARNING  Process 64489 had been terminated.


[2024-08-30 15:45:42 +1000][promptflow._sdk._orchestrator.run_submitter][WARNING] - 4 out of 4 runs failed in batch run.
 Please check out /home/krbock/.promptflow/.runs/app_target_modelendpoints_75ml9r66_20240830_154535_138482 for more details.
[2024-08-30 15:45:42 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Uploading run 'app_target_modelendpoints_75ml9r66_20240830_154535_138482' to cloud...
[2024-08-30 15:45:55 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Updating run 'app_target_modelendpoints_75ml9r66_20240830_154535_138482' portal url to 'https://ai.azure.com/projectflows/trace/run/app_target_modelendpoints_75ml9r66_20240830_154535_138482/details?wsid=/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourcegroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635'.


Portal url: https://ai.azure.com/projectflows/trace/run/app_target_modelendpoints_75ml9r66_20240830_154535_138482/details?wsid=/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourcegroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635
2024-08-30 15:45:38 +1000   57173 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-08-30 15:45:38 +1000   57173 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2024-08-30 15:45:38 +1000   57173 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 4}.
2024-08-30 15:45:40 +1000   57173 execution.bulk     INFO     Process name(ForkProcess-36:1)-Process id(64483)-Line number(0) start execution.
2024-08-30 15:45:40 +1000   57

ValueError: Missing required inputs for evaluator content_safety : ['answer'].

View the results

In [ ]:
pprint(results)

In [ ]:
pd.DataFrame(results["rows"])